In [12]:
import datetime
import os
from typing import Callable, Optional
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch

In [13]:
pd.set_option('display.max_columns', None)
path = './archive/HI-Small_Trans.csv'
#path = './archive/LI-Small_Trans.csv'
df = pd.read_csv(path)

# 데이터확인
-> 우리의 피쳐 엔지니어링은 더 심오하다!
* 계정을 노드로, 거래를 에지로 간주하여 전체 데이터 세트를 노드 분류 문제로 변환할
* 의심스러운 계정을 분류하기 위해 모든 거래에서 수신자와 지불자의 모든 계정을 추출

In [14]:
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [15]:
df.info

<bound method DataFrame.info of                 Timestamp  From Bank    Account  To Bank  Account.1  \
0        2022/09/01 00:20         10  8000EBD30       10  8000EBD30   
1        2022/09/01 00:20       3208  8000F4580        1  8000F5340   
2        2022/09/01 00:00       3209  8000F4670     3209  8000F4670   
3        2022/09/01 00:02         12  8000F5030       12  8000F5030   
4        2022/09/01 00:06         10  8000F5200       10  8000F5200   
...                   ...        ...        ...      ...        ...   
5078340  2022/09/10 23:57      54219  8148A6631   256398  8148A8711   
5078341  2022/09/10 23:35         15  8148A8671   256398  8148A8711   
5078342  2022/09/10 23:52     154365  8148A6771   256398  8148A8711   
5078343  2022/09/10 23:46     256398  8148A6311   256398  8148A8711   
5078344  2022/09/10 23:37     154518  8148A6091   256398  8148A8711   

         Amount Received Receiving Currency   Amount Paid Payment Currency  \
0            3697.340000          US 

In [16]:
df.dtypes

Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object

In [17]:
df.isnull().sum()

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64

각 거래의 지불 및 수신 금액을 나타내는 두 개의 열이 있는데, 서로 다른 통화 간의 거래 수수료/거래가 아닌 한, 같은 값을 공유할 때 금액을 두 개의 열로 분할해야 하는지.

In [18]:
print('Amount Received equals to Amount Paid:')
print(df['Amount Received'].equals(df['Amount Paid']))
print('Receiving Currency equals to Payment Currency:')
print(df['Receiving Currency'].equals(df['Payment Currency']))

Amount Received equals to Amount Paid:
False
Receiving Currency equals to Payment Currency:
False


* 다양한 거래가 포함 되어 있다고.
* Currency가 틀리면, 입출금 되는 금액이 틀릴 수 있지.

In [19]:
not_equal1 = df.loc[~(df['Amount Received'] == df['Amount Paid'])]  # ~ not
print(not_equal1)

                Timestamp  From Bank    Account  To Bank  Account.1  \
1173     2022/09/01 00:22       1362  80030A870     1362  80030A870   
7156     2022/09/01 00:28      11318  800C51010    11318  800C51010   
7925     2022/09/01 00:12        795  800D98770      795  800D98770   
8467     2022/09/01 00:01       1047  800E92CF0     1047  800E92CF0   
11529    2022/09/01 00:22      11157  80135FFC0    11157  80135FFC0   
...                   ...        ...        ...      ...        ...   
5078167  2022/09/10 23:30      23537  803949A90    23537  803949A90   
5078234  2022/09/10 23:59      16163  803638A90    16163  803638A90   
5078236  2022/09/10 23:55      16163  803638A90    16163  803638A90   
5078316  2022/09/10 23:44     215064  808F06E11   215064  808F06E10   
5078318  2022/09/10 23:45     215064  808F06E11   215064  808F06E10   

         Amount Received Receiving Currency  Amount Paid Payment Currency  \
1173           52.110000               Euro        61.06        US Dol

In [20]:
not_equal2 = df.loc[~(df['Receiving Currency'] == df['Payment Currency'])] # ~ not
print(not_equal2)

                Timestamp  From Bank    Account  To Bank  Account.1  \
1173     2022/09/01 00:22       1362  80030A870     1362  80030A870   
7156     2022/09/01 00:28      11318  800C51010    11318  800C51010   
7925     2022/09/01 00:12        795  800D98770      795  800D98770   
8467     2022/09/01 00:01       1047  800E92CF0     1047  800E92CF0   
11529    2022/09/01 00:22      11157  80135FFC0    11157  80135FFC0   
...                   ...        ...        ...      ...        ...   
5078167  2022/09/10 23:30      23537  803949A90    23537  803949A90   
5078234  2022/09/10 23:59      16163  803638A90    16163  803638A90   
5078236  2022/09/10 23:55      16163  803638A90    16163  803638A90   
5078316  2022/09/10 23:44     215064  808F06E11   215064  808F06E10   
5078318  2022/09/10 23:45     215064  808F06E11   215064  808F06E10   

         Amount Received Receiving Currency  Amount Paid Payment Currency  \
1173           52.110000               Euro        61.06        US Dol

In [21]:
print(72158+72170)

144328


* 두 df의 데이터는 거래 수수료와 다른 통화 간의 거래가 있으므로, 금액 열을 결합/삭제할 수 없음.

* 열을 인코딩할 것이므로 동일한 속성의 클래스가 정렬되어 있는지 확인해야 함. 
* 수신 통화와 지불 통화 목록이 동일한지 확인 -> 동일

In [22]:
sorted(df['Receiving Currency'].unique())

['Australian Dollar',
 'Bitcoin',
 'Brazil Real',
 'Canadian Dollar',
 'Euro',
 'Mexican Peso',
 'Ruble',
 'Rupee',
 'Saudi Riyal',
 'Shekel',
 'Swiss Franc',
 'UK Pound',
 'US Dollar',
 'Yen',
 'Yuan']

In [23]:
sorted(df['Payment Currency'].unique())

['Australian Dollar',
 'Bitcoin',
 'Brazil Real',
 'Canadian Dollar',
 'Euro',
 'Mexican Peso',
 'Ruble',
 'Rupee',
 'Saudi Riyal',
 'Shekel',
 'Swiss Franc',
 'UK Pound',
 'US Dollar',
 'Yen',
 'Yuan']

# 데이터 전처리

- PyG 데이터셋에서 사용되는 함수를 소개
- 데이터셋과 모델 학습은 하단 섹션에서 제공

1. 최소 최대 정규화로 타임스탬프를 변환.
2. 계좌 번호와 함께 은행 코드를 추가하여 각 계좌에 대한 고유 ID를 만듦.
3. 수신 계좌, 수신 금액 및 통화의 정보로 receiving_df를 만듦.
4. 지불자 계좌, 지불 금액 및 통화의 정보로 paying_df를 만듦.
5. 모든 거래에서 사용된 통화 목록을 만듦.
6. sklearn LabelEncoder로 클래스별로 'Payment Format', 'Payment Currency', 'Receiving Currency'에 레이블을 지정.

In [24]:
def df_label_encoder(df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df

def preprocess(df):
        df = df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min()) # Timestamp 를 0~1 사이 값으로 정규화 함

        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']
        df = df.sort_values(by=['Account'])  ## by 정렬의 기준이 되는 열을 지정

        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]

        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)
        
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

In [25]:
df, receiving_df, paying_df, currency_ls = preprocess(df = df)

In [26]:
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
4278714,0.456320,10057,10057_803A115E0,29467,29467_803E020C0,787197.11,13,787197.11,13,3,0
2798190,0.285018,10057,10057_803A115E0,29467,29467_803E020C0,787197.11,13,787197.11,13,3,0
2798191,0.284233,10057,10057_803A115E0,29467,29467_803E020C0,681262.19,13,681262.19,13,4,0
3918769,0.417079,10057,10057_803A115E0,29467,29467_803E020C0,681262.19,13,681262.19,13,4,0
213094,0.000746,10057,10057_803A115E0,10057,10057_803A115E0,146954.27,13,146954.27,13,5,0


In [29]:
display(receiving_df.head())
display(paying_df.head())
display(currency_ls)

,Account,Amount Received,Receiving Currency
4278714,29467_803E020C0,787197.11,13
2798190,29467_803E020C0,787197.11,13
2798191,29467_803E020C0,681262.19,13
3918769,29467_803E020C0,681262.19,13
213094,10057_803A115E0,146954.27,13


,Account,Amount Paid,Payment Currency
4278714,10057_803A115E0,787197.11,13
2798190,10057_803A115E0,787197.11,13
2798191,10057_803A115E0,681262.19,13
3918769,10057_803A115E0,681262.19,13
213094,10057_803A115E0,146954.27,13


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

# 데이터 전처리

In [ ]:
|